Text Preprocessing
--------
Text preprocessing made on the competition datasets.
The preprocessing consists of 4 steps:

 1. **Removing tags and URIs from contents**
 2. **Removing punctuation from titles and contents**
 3. **Removing stopwords from titles and contents**
 4. **Converting the tags from string to a list of tags**

This type of operations can be used as a first step for any other process regarding the competition.

In [47]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
import string
import os
import itertools

Datasets loading
---------

In [20]:
dataframes = {
    "cooking": pd.read_csv("./input/cooking.csv"),
    "crypto": pd.read_csv("./input/crypto.csv"),
    "robotics": pd.read_csv("./input/robotics.csv"),
    "biology": pd.read_csv("./input/biology.csv"),
    "travel": pd.read_csv("./input/travel.csv"),
    "diy": pd.read_csv("./input/diy.csv"),
}

For simplicity, i'll show an example of the steps of the preprocessing on an item of the robotics dataset

In [21]:
print(dataframes["robotics"].iloc[1])

id                                                         2
title      How can I modify a low cost hobby servo to run...
content    <p>I've got some hobby servos (<a href="http:/...
tags                                         control rcservo
Name: 1, dtype: object


Removing html tags and uris from contents
-----------

In [22]:
uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTagsAndUris(x):
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""

In [23]:
# This could take a while
for df in dataframes.values():
    df["content"] = df["content"].map(stripTagsAndUris)

In [24]:
print(dataframes["robotics"].iloc[1])

id                                                         2
title      How can I modify a low cost hobby servo to run...
content    I've got some hobby servos (Power HD 1501MGs) ...
tags                                         control rcservo
Name: 1, dtype: object


Removing punctuation from titles and contents
-----------

In [25]:
def removePunctuation(x):
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

In [26]:
for df in dataframes.values():
    df["title"] = df["title"].map(removePunctuation)
    df["content"] = df["content"].map(removePunctuation)

In [27]:
print(dataframes["robotics"].iloc[1])

id                                                         2
title      how can i modify a low cost hobby servo to run...
content    i ve got some hobby servos  power hd 1501mgs  ...
tags                                         control rcservo
Name: 1, dtype: object


Removing stopwords from titles and contents
-----------

In [35]:
stops = set(stopwords.words("english"))
numberRegex = re.compile(r"^\d+(th|rd|nd)?$")

def shouldInclude(word):
    if word in stops:           # Skip stopwords from NLTK
        return False
    if numberRegex.match(word) is not None: # Skip words like "2" or "3rd"
        return False
    if word[0] == '\\':         # Skip LaTeX commands beginning with \
        return False
    if len(word) <= 1:
        return False

    return True

def removeStopwords(x):
    # Removing all the stopwords
    filtered_words = [word for word in x.split() if shouldInclude(word)]
    return " ".join(filtered_words)

In [36]:
for df in dataframes.values():
    df["title"] = df["title"].map(removeStopwords)
    df["content"] = df["content"].map(removeStopwords)

In [37]:
print(dataframes["robotics"].iloc[1])

id                                                         2
title                 modify low cost hobby servo run freely
content    got hobby servos power hd 1501mgs like able co...
tags                                      [control, rcservo]
Name: 1, dtype: object


Splitting tags string in a list of tags
-----------

In [38]:
for df in dataframes.values():
    # From a string sequence of tags to a list of tags
    df["tags"] = df["tags"].map(lambda x: x.split())

AttributeError: 'list' object has no attribute 'split'

In [32]:
print(dataframes["robotics"].iloc[1])

id                                                         2
title                 modify low cost hobby servo run freely
content    got hobby servos power hd 1501mgs like able co...
tags                                      [control, rcservo]
Name: 1, dtype: object


Saving preprocessed dataframes to csv
-----------

In [39]:
for name, df in dataframes.items():
    # Saving to file
    df.to_csv(os.path.join(".", "preprocessed-input", "data-" + name, name + "_light.csv"), index=False)

In [46]:
for name, df in dataframes.items():
    with open(os.path.join(".", "preprocessed-input", "data-" + name, name + "_gibbs.txt"), "w") as f:
        f.write("{}\n".format(df.shape[0]))
        for line in df.iterrows():
            f.write(line[1]['title'] + " " + line[1]['content'] + "\n")

In [48]:
def process_lda_results():
    for name, df in dataframes.items():
        with open(os.path.join(".", "preprocessed-input", "data-" + name, "model-final.tassign"), "r") as f:
            tag_topics = {}
            for question, tassign in itertools.izip(df.iterrows(), f):
                for topic, in tassign.split(" "):
